# Taller: Análisis de Tweets con Gemini API
Autor: _(tu nombre)_

## Objetivos
1. Configurar la API de Google Generative AI (Gemini).
2. Limpiar y pre‑procesar un corpus de tweets políticos.
3. Clasificar sentimiento con Gemini.
4. Extraer temas con LDA y nombrarlos con Gemini.
5. Segmentar usuarios y generar una micro‑campaña basada en insights.

**Dataset**: `tweets_partidos.csv` (columnas: `cuenta`, `partido`, `timestamp`, `tweet`).

In [11]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/SegundoCorte/tweets_politica_kaggle.csv.zip

--2025-05-08 01:03:30--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/SegundoCorte/tweets_politica_kaggle.csv.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/SegundoCorte/tweets_politica_kaggle.csv.zip [following]
--2025-05-08 01:03:30--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/SegundoCorte/tweets_politica_kaggle.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18276075 (17M) [application/zip]
Saving to: ‘tweets_politica_kaggle.csv.zip.1’

t

In [12]:
!unzip tweets_politica_kaggle.csv.zip

Archive:  tweets_politica_kaggle.csv.zip
  inflating: tweets_politica_kaggle.csv  


In [13]:
!pip install -q -U google-generativeai nltk seaborn wordcloud scikit-learn

In [14]:
data=pd.read_csv('tweets_politica_kaggle.csv', delimiter="\t",on_bad_lines='skip')

### 🔍 Preguntas – Sección 1 (Exploración)
1. **¿Cuántos tweets hay en total?**  
2. **¿Qué partidos aparecen y cuántos tweets aporta cada uno?**  
3. **¿Cuál es el rango de fechas cubierto por los tweets?**  
4. **¿Qué partido genera más conversación y por qué crees que ocurre?**

In [17]:
# prompt: ¿Cuántos tweets hay en total?
# ¿Qué partidos aparecen y cuántos tweets aporta cada uno?
# ¿Cuál es el rango de fechas cubierto por los tweets?
# ¿Qué partido genera más conversación y por qué crees que ocurre?¿Cuántos tweets hay en total?
# ¿Qué partidos aparecen y cuántos tweets aporta cada uno?
# ¿Cuál es el rango de fechas cubierto por los tweets?
# ¿Qué partido genera más conversación y por qué crees que ocurre?

import pandas as pd

# Load the dataframe.
data = pd.read_csv('tweets_politica_kaggle.csv', delimiter="\t", on_bad_lines='skip')

# 1. ¿Cuántos tweets hay en total?
total_tweets = len(data)
print(f"1. Número total de tweets: {total_tweets}")

# 2. ¿Qué partidos aparecen y cuántos tweets aporta cada uno?
tweets_per_party = data['partido'].value_counts()
print("\n2. Tweets por partido:")
print(tweets_per_party)

# 3. ¿Cuál es el rango de fechas cubierto por los tweets?
data['timestamp'] = pd.to_datetime(data['timestamp'])
min_date = data['timestamp'].min()
max_date = data['timestamp'].max()
print(f"\n3. Rango de fechas: {min_date.date()} - {max_date.date()}")

# 4. ¿Qué partido genera más conversación y por qué crees que ocurre?
most_talked_party = tweets_per_party.index[0]
print(f"\n4. Partido con más conversación: {most_talked_party}")
print("Posibles razones (requiere análisis más profundo):")
print("- Mayor actividad en redes sociales del partido.")
print("- Temas controversiales o relevantes para la conversación pública.")
print("- Mayor cantidad de seguidores o alcance en las redes sociales.")


1. Número total de tweets: 181504

2. Tweets por partido:
partido
psoe          42818
vox           38277
pp            35059
podemos       34442
ciudadanos    30908
Name: count, dtype: int64

3. Rango de fechas: 1970-01-01 - 1970-01-01

4. Partido con más conversación: psoe
Posibles razones (requiere análisis más profundo):
- Mayor actividad en redes sociales del partido.
- Temas controversiales o relevantes para la conversación pública.
- Mayor cantidad de seguidores o alcance en las redes sociales.


### 🧹 Preguntas – Sección 2 (Limpieza)
5. Explica **por qué es importante limpiar y normalizar el texto**.  
6. Enumera **tres tipos de “ruido”** que removes y da un ejemplo de cada uno.

In [22]:
import google.generativeai as genai

# Configura tu API key (ya debes haberla cargado antes)
genai.configure(api_key="TU_API_KEY")

# Carga el modelo (esto es lo que falta en tu código)
model_fast = genai.GenerativeModel('gemini-pro')

# Ahora puedes usar model_fast en tu función
def classify_sentiment(text, model=model_fast):
    prompt = (f"Clasifica el sentimiento del siguiente tweet como 'positivo', "
              f"'neutral' o 'negativo'. Solo responde con una palabra.\n\nTweet:\n{text}")
    return model.generate_content(prompt).text.strip().lower()


In [23]:
def classify_sentiment(text, model=model_fast):
    prompt = (f"Clasifica el sentimiento del siguiente tweet como 'positivo', "
              f"'neutral' o 'negativo'. Solo responde con una palabra.\n\nTweet:\n{text}")
    return model.generate_content(prompt).text.strip().lower()

In [29]:
dc = df.head(5)
dc['sentiment'] = dc['tweet'].apply(classify_sentiment)
dc.head()


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.

### 😊 Preguntas – Sección 3 (Sentimiento)
7. Presenta la **distribución global** de sentimientos y comenta.  
8. **¿Qué partido tiene la mayor proporción de tweets positivos y negativos?**  
9. Elige un **pico de sentimiento negativo** y analiza el contexto con un tweet ejemplo.

### 🗂️ Preguntas – Sección 4 (Temas)
10. Lista los **nombres de los temas** generados. ¿Alguno es inesperado?  
11. Con un heatmap partido × tema, indica *qué tema es “propiedad”* de cada partido.  
12. Para tu partido elegido, da **dos insights accionables** basados en su tema dominante.

### 👥 Preguntas – Sección 5 (Segmentación)
13. Describe cada **cluster** en una frase (actividad y tono).  
14. **¿Qué segmento priorizarías** para viralizar un mensaje y por qué?  
15. Propón **una acción de engagement** distinta para cada segmento.

In [ ]:
def generar_copy(partido, topic, tono='inspirador'):
    prompt = (f"Actúa como community manager del partido {partido}. "
              f"Crea un tweet de máx. 250 caracteres sobre el tema '{topic}'. "
              f"Tono {tono}. No incluyas hashtags ni menciones.")
    return model.generate_content(prompt).text.strip()


### 📝 Preguntas – Sección 6 (Micro‑campaña)
16. Presenta tus **tres tweets** generados.  
17. Justifica:  
   a) **Tema** elegido.  
   b) **Tono** y **horario** óptimos.  
18. Define un **KPI de éxito** y la meta para la campaña.

## Próximos pasos
1. Analiza la hora de publicación (`df['fecha'].dt.hour`) para programar los tweets.
2. Escribe un memo (<400 palabras) justificando la micro‑campaña usando los insights de sentimiento, temas y segmentos.
3. Exporta notebook ejecutado y memo para entrega.